# Sequence Labelling

Jusque là, nous avons employé des méthodes supervisées pour classer des éléments les uns après les autres en fonctions de leurs caractéristiques. Cependant, les points de données ne sont pas toujours indépendants, en particulier lorsqu'on aborde la langue : ils font partie de séquences (traitement de la parole, parties du discours, entités nommées).

Les modèles que nous employés jusque là ne peuvent traiter les séquences. Les algorithmes classiques pour l'annotation de séquences sont les Hidden Markov Models (HMM) et les Conditional Random Field classifiers (CRF). Les réseaux de neurones (type LSTM) et Transformers sont les architectures les plus modernes pour cette tâche.

## CRF

Contrairement à HMM, CRF repose sur des features extraites pour chaque élément. Chaque feature est en interne associée à une fonction, comme ci dessous (pseudo-code):

* f = 1 si mot termine par "-re" sinon f=0
* f = 1 si mot commence "-re" et tag-1 est "NOM" sinon 0

Chaque fonction est associée à un poids, dont la valeur est définie aléatoirement au départ. Les valeurs des poids sont mis à jours à chaque tour de l'entraînement. Pour cela, on emploie des algorithmes tels que Gradient Descent. A partir des résultats de chaque fonction multiplié par leur poids respectif, le modèle détermine la probabilité qu'une unité appartienne à une certaine catégorie.  

Les CRF généraux peuvent prendre en compte tout élément précédent ou suivante de la séquence. Ils sont cependant complexes à entraîner. Les CRF linéaires quant à eux ne prennent en compte que l'élément précédent de la séquence, et sont donc plus simples à entraîner, d'autant plus que des tâches comme le POS tagging ou la NER ne nécessite pas de prendre en compte des contextes plus grands.

CRFSuite implémente les CRF linéaires.

Dans ce cours, nous allons employer le CRF, CRF étant en général meilleur que HMM. Pour cela, nous allons employer la librairie "sklearn-crfsuite". Bien que ne faisant pas partie de l'écosystème scikit-learn, elle propose la même interface.

In [3]:
!pip install sklearn-crfsuite
# il y a un bug avec l'implémentation de sklearn-crfsuite avec sklearn qui retourne une AttributeError 
# à la fin de l'entraînement. On utilise le try...except pour éviter l'erreur. 
# une autre solution est d'installer une version de sklearn inférieure à 0.24

# !pip install -U 'scikit-learn<0.24'

## POS tagging

Nous allons entraîner un modèle CRF pour l'analyse en partie du discours en espagnol. Pour cela, nous allons utiliser le dataset "COnLL 2002", qui est entre autre disponible via NLTK

In [14]:
import nltk


In [15]:

nltk.download('conll2002')


[nltk_data] Downloading package conll2002 to
[nltk_data]     /Users/nicolasgutehrle/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


True

In [16]:
nltk.corpus.conll2002.fileids()

['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [17]:
train = list(nltk.corpus.conll2002.iob_sents('esp.train'))
dev = list(nltk.corpus.conll2002.iob_sents('esp.testa'))
test = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

## Format BIO

Les jeux de données pour le POS tagging ou la NER sont souvent au format conll, qui est similaire au CSV, et où chaque ligne est un token, et chaque colonne présente des informations linguistiques ou une annotation. 

Les POS ne peuvent concerner qu'un seul token à la fois. Cependant les entités nommées peuvent comprendre plusieurs tokens. Pour annoter, on emploi souvent le format BIO (Beginning, Inside, Outside) :
* le début d'une entité est noté B, suivi généralement par le type d'entité (ex: B-LOC, B-PER)
* tous les autres tokens contenus dans cette entité sont annotés I, suivi du type de l'entité (ex: I-LOC, I-PER)
* tout autre token ne faisant pas partie d'une entité est noté O 

In [8]:
train[2]

[('El', 'DA', 'O'),
 ('Abogado', 'NC', 'B-PER'),
 ('General', 'AQ', 'I-PER'),
 ('del', 'SP', 'I-PER'),
 ('Estado', 'NC', 'I-PER'),
 (',', 'Fc', 'O'),
 ('Daryl', 'VMI', 'B-PER'),
 ('Williams', 'NC', 'I-PER'),
 (',', 'Fc', 'O'),
 ('subrayó', 'VMI', 'O'),
 ('hoy', 'RG', 'O'),
 ('la', 'DA', 'O'),
 ('necesidad', 'NC', 'O'),
 ('de', 'SP', 'O'),
 ('tomar', 'VMN', 'O'),
 ('medidas', 'NC', 'O'),
 ('para', 'SP', 'O'),
 ('proteger', 'VMN', 'O'),
 ('al', 'SP', 'O'),
 ('sistema', 'NC', 'O'),
 ('judicial', 'AQ', 'O'),
 ('australiano', 'AQ', 'O'),
 ('frente', 'RG', 'O'),
 ('a', 'SP', 'O'),
 ('una', 'DI', 'O'),
 ('página', 'NC', 'O'),
 ('de', 'SP', 'O'),
 ('internet', 'NC', 'O'),
 ('que', 'PR', 'O'),
 ('imposibilita', 'VMI', 'O'),
 ('el', 'DA', 'O'),
 ('cumplimiento', 'NC', 'O'),
 ('de', 'SP', 'O'),
 ('los', 'DA', 'O'),
 ('principios', 'NC', 'O'),
 ('básicos', 'AQ', 'O'),
 ('de', 'SP', 'O'),
 ('la', 'DA', 'O'),
 ('Ley', 'NC', 'B-MISC'),
 ('.', 'Fp', 'O')]

Notre objectif est d'extraire les features pour chaque token, puis d'entraîner un modèle et de le tester. Contrairement aux précédents modèles de classification que nous avons vu, nous aurons besoin d'accéder aux features des tokens précédents. 

In [18]:
from typing import List

def word2features(sent:List[tuple[str]], i:int) -> dict:
    """
    Extraie les features pour le mot actuel de la phrase, et si possible, des mots d'avant et d'après
    Puisque l'on entraîne un linear-CRF, il n'y a aucun intérêt à extraire des features au delà d'un mot
    d'avant ou d'après

    :param sent: Liste de phrases contenant les tokens
    :type sent: List[tuple[str]]
    :param i: index du mot
    :type i: int
    :return: dictionnaire de features du mot
    :rtype: dict
    """
    word = sent[i][0]

    features = {
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        # features du mot suivant
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.isdigit()': word1.isdigit()
        })
    else:
        # BOS : beginning of sentence : le mot est le premier de la phrase
        features['BOS'] = True

    if i < len(sent)-1:
        # features du mot précédent
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:word.isdigit()': word1.isdigit()
        })
    else:
        # EOS : end of sentence : le mot est le dernier de la phrase
        features['EOS'] = True

    return features


In [19]:
def sent2labels(sent:List[tuple[str]], type:str="pos") -> List[str]:
    """
    Extraie les classes de chaque phrase (soit POS, soit NER)

    :param sent: liste de phrase à traiter
    :type sent: List[tuple[str]]
    :param type: type de classe à extraire, soit "pos" soit "ner", defaults to "pos"
    :type type: str, optional
    :return: liste des classes
    :rtype: List[str]
    """
    if type == "pos":
        return [postag for token, postag, label in sent]
    elif type == "ner":
        return [label for token, postag, label in sent]

def sent2features(sent:List[List[tuple[str]]]) -> List[dict]:
    """
    Simple fonction pour extraire les features des mots d'une phrase

    :param sent: liste de phrase à traiter
    :type sent: List[List[tuple[str]]]
    :return: liste des features pour chaque mot
    :rtype: List[dict]
    """
    return [word2features(sent, i) for i in range(len(sent))]

def processPipeline(sent:List[List[tuple[str]]], type='pos') -> tuple[List[List[dict]], List[List[str]]]:
    """
    Transforme un jeu de données au format requis, en extrayant les features et les classes

    :param sent: Liste de phrases à traiter
    :type sent: List[List[tuple[str]]]
    :param type: Typle de classe à extraire, soit "pos" soit "ner", defaults to 'pos'
    :type type: str, optional
    :return: Tuple contenant X puis y
    :rtype: tuple[List[List[dict]], List[List[str]]]
    """
    X = [sent2features(x) for x in sent]
    y = [sent2labels(x) for x in sent]

    return X, y

In [20]:
X_train, y_train = processPipeline(train, type="pos")
X_dev, y_dev = processPipeline(dev, type="pos")
X_test, y_test = processPipeline(test, type="pos")

In [21]:
X_train[0]

[{'word.lower()': 'melbourne',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'BOS': True,
  '+1:word.lower()': '(',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.isdigit()': False},
 {'word.lower()': '(',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  '-1:word.lower()': 'melbourne',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-1:word.isdigit()': False,
  '+1:word.lower()': 'australia',
  '+1:word.istitle()': True,
  '+1:word.isupper()': False,
  '+1:word.isdigit()': False},
 {'word.lower()': 'australia',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  '-1:word.lower()': '(',
  '-1:word.istitle()': False,
  '-1:word.isupper()': False,
  '-1:word.isdigit()': False,
  '+1:word.lower()': ')',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.isdigit()': False},
 {'word.lower()': ')',
  'word.isupper()': False,
  'word.isti

## Entraînement du modèle

Les hyperparamètres principaux sont :
* algorithm : le choix de l'algorithme pour mettre à jour les poids. Par défaut, l'algorithme est basé sur Gradient Descent
* min_freq : occurrence minimale d'une feature. Par défaut à 0
* all_possible_states : Génère ou non des valeurs pour features qui n'existent pas dans le dataset. Peut améliorer les performances du modèle mais augmente le temps d'entraînement. Par défaut sur False.
* all_possible_transitions : Génère ou non des transitions qui n'existent pas dans le dataset. De même, peut améliorer les performances du modèle mais augmente le temps d'entraînement. Par défaut sur False.
* c1 : valeur pour la régularisation L1
* c2 : valeur pour la régularisation L2

La régularisation L1 aide le modèle à sélectionner des features plus pertinentes (en particulier s'il y a beaucoup de features), tandis que L2 aide le modèle à généraliser. 

Ci-dessous, nous entraîner un modèle CRF linéaire avec les valeurs par défaut pour chaque hyperparamètre.

Documentation du modèle : https://sklearn-crfsuite.readthedocs.io/en/latest/api.html

In [13]:
y_train

[['NP', 'Fpa', 'NP', 'Fpt', 'Fc', 'Z', 'NC', 'Fpa', 'NC', 'Fpt', 'Fp'],
 ['Fg'],
 ['DA',
  'NC',
  'AQ',
  'SP',
  'NC',
  'Fc',
  'VMI',
  'NC',
  'Fc',
  'VMI',
  'RG',
  'DA',
  'NC',
  'SP',
  'VMN',
  'NC',
  'SP',
  'VMN',
  'SP',
  'NC',
  'AQ',
  'AQ',
  'RG',
  'SP',
  'DI',
  'NC',
  'SP',
  'NC',
  'PR',
  'VMI',
  'DA',
  'NC',
  'SP',
  'DA',
  'NC',
  'AQ',
  'SP',
  'DA',
  'NC',
  'Fp'],
 ['DA',
  'NC',
  'SP',
  'NC',
  'AQ',
  'VMI',
  'NC',
  'RG',
  'SP',
  'CS',
  'DI',
  'NC',
  'SP',
  'NC',
  'AQ',
  'SP',
  'NC',
  'SP',
  'NC',
  'Fpa',
  'NP',
  'Fpt',
  'P0',
  'VMS',
  'AQ',
  'SP',
  'VMN',
  'DI',
  'NC',
  'AQ',
  'CC',
  'VMN',
  'DA',
  'NC',
  'SP',
  'DA',
  'NC',
  'SP',
  'DA',
  'NC',
  'SP',
  'CS',
  'DA',
  'NC',
  'PR',
  'PP',
  'VMI',
  'VMI',
  'VAN',
  'VMP',
  'NC',
  'SP',
  'DA',
  'VMP',
  'SP',
  'NC',
  'SP',
  'DA',
  'NC',
  'AQ',
  'Fp'],
 ['DD',
  'NC',
  'AQ',
  'VMI',
  'DI',
  'NC',
  'SP',
  'NC',
  'Fc',
  'NC',
  'SP',
  'D

In [22]:
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF()

crf.fit(X_train, y_train)

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

## Evaluation

sklearn-crfsuite met à disposition plusieurs métriques pour évaluer le modèle

Documentation : https://sklearn-crfsuite.readthedocs.io/en/latest/api.html#module-sklearn_crfsuite.metrics

In [23]:
from sklearn_crfsuite.metrics import flat_precision_score, flat_recall_score, flat_f1_score, flat_classification_report

dev_pred = crf.predict(X_dev)
print('P :', flat_precision_score(y_dev, dev_pred, average='micro'))
print('R :', flat_recall_score(y_dev, dev_pred, average='micro'))
print('F1 :', flat_f1_score(y_dev, dev_pred, average='micro'))

P : 0.9247208208151465
R : 0.9247208208151465
F1 : 0.9247208208151465


In [36]:
y_dev

[['NC', 'VMI', 'Fpa', 'NC', 'Fpt', 'Fc', 'Z', 'NC', 'Fpa', 'NP', 'Fpt', 'Fp'],
 ['Fg'],
 ['DA',
  'NC',
  'AQ',
  'AQ',
  'VAI',
  'VMP',
  'DI',
  'NC',
  'AQ',
  'SP',
  'VMN',
  'SP',
  'NC',
  'DN',
  'NC',
  'SP',
  'AQ',
  'NC',
  'SP',
  'DA',
  'NC',
  'AQ',
  'SP',
  'NC',
  'VMI',
  'SP',
  'PR',
  'VMI',
  'DA',
  'NC',
  'SP',
  'DA',
  'NC',
  'AQ',
  'VMI',
  'Z',
  'NC',
  'Fc',
  'VMI',
  'RG',
  'DA',
  'NC',
  'SP',
  'AQ',
  'NC',
  'NC',
  'Fc',
  'NC',
  'AQ',
  'AQ',
  'Fp'],
 ['SP',
  'DI',
  'NC',
  'PR',
  'VMI',
  'SP',
  'DA',
  'NC',
  'SP',
  'AQ',
  'SP',
  'NC',
  'VMI',
  'SP',
  'NC',
  'SP',
  'Z',
  'Fc',
  'NC',
  'VMI',
  'CS',
  'DA',
  'NC',
  'VAI',
  'VMP',
  'DI',
  'DA',
  'NC',
  'AQ',
  'SP',
  'DA',
  'NC',
  'AQ',
  'SP',
  'NC',
  'Fc',
  'DA',
  'NC',
  'AQ',
  'SP',
  'NC',
  'Fpa',
  'NP',
  'Fpt',
  'Fc',
  'SP',
  'RG',
  'SP',
  'NC',
  'CC',
  'NC',
  'SP',
  'NC',
  'VMP',
  'Fp'],
 ['Fe',
  'AQ',
  'VMI',
  'DI',
  'NC',
  'SP',


In [24]:
test_pred = crf.predict(X_test)
print('P :', flat_precision_score(y_test, test_pred, average='micro'))
print('R :', flat_recall_score(y_test, test_pred, average='micro'))
print('F1 :', flat_f1_score(y_test, test_pred, average='micro'))

P : 0.9357305027846234
R : 0.9357305027846234
F1 : 0.9357305027846234


## Note sur l'évaluation Précision, Rappel, F1

Ci-dessous, on précise "average=micro". Les mesures de Précision, Rappel et F1 sont prévues pour évaluer des catégories binaires. Lorsqu'on a plus de deux catégories, il faut évaluer chaque catégorie contre les autres (one-vs-all) puis faire la moyenne de chaque évaluation. Il y a plusieurs façons de faire cette moyenne :

* macro : fait la moyenne des évaluation, sans prendre en compte leur distribution dans le jeu de dataset
* micro : fait la moyenne des évaluation après avoir ajouté un poids à chaque classe de telles sortes qu'elles contribuent de manière égale au score final
* weighted : fait la moyenne des évaluation après avoir ajouté un poids pour chaque classe en fonction de son support (distribution dans le jeu de données), pour que les classes les moins représentées aient un poids plus important

## GridSearch

In [25]:
from sklearn.model_selection import RandomizedSearchCV

param_space = {
    'c1': [0.1, 0.01, 0.001],
    'c2': [0.1, 0.01, 0.001],
    'all_possible_transitions': [True, False],
    'all_possible_states': [True, False]
    
}
clf = RandomizedSearchCV(sklearn_crfsuite.CRF(), param_space, random_state=42)
clf.fit(X_train, y_train)

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

# Ressources

* Documentation : https://sklearn-crfsuite.readthedocs.io/en/latest/
* Jurafsky et al : https://web.stanford.edu/~jurafsky/slp3/8.pdf
* https://www.youtube.com/watch?v=rI3DQS0P2fk&t=1144s&ab_channel=ritvikmath